In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import warnings
import time
warnings.filterwarnings('ignore')
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from functools import partial

Возьмем код и результаты полученные в работе 5.2 во втором задании.

In [5]:
df = pd.read_csv('diabetes.csv')
X = df.drop('Outcome', axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [40]:
bst = XGBClassifier(learning_rate=0.1, max_depth=2, n_estimators=150, subsample=0.8, reg_lambda=4, random_state=13)

start_time = time.time()
bst.fit(X_train, y_train)
bst_time = time.time() - start_time
bst_pred = bst.predict(X_test)
print("Accuracy:", accuracy_score(y_test, bst_pred))
print("Время обучения:", bst_time)

Accuracy: 0.8246753246753247
Время обучения: 0.04382610321044922


Используя библиотеку Scikit-Learn для XGBoost выполним процесс подбора гиперпараметров используя Random Search

In [31]:
param_dist = {'learning_rate': [0.01, 0.05, 0.1, 0.2], 'max_depth': [2, 3, 4, 5, 6, 7], 'n_estimators': [50, 100, 150, 200],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0], 'reg_lamda': [10, 30, 60, 80, 100], 'reg_alpha': [10, 30, 70, 110]}

xgb = XGBClassifier(random_state=13)
start_time = time.time()
random_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_dist, n_iter=50, cv=5,
                                   scoring='accuracy', random_state=13)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшая точность:", random_search.best_score_)
print("Время:", time.time() - start_time)

Лучшие параметры: {'subsample': 0.7, 'reg_lamda': 60, 'reg_alpha': 10, 'n_estimators': 150, 'max_depth': 4, 'learning_rate': 0.1}
Лучшая точность: 0.7638544582167134
Время: 7.932732820510864


Выполним подбор гиперпараметров, для той же модели используя алгоритм TPE и библиотеку Hyperopt

In [44]:
def objective(model, params):
    model.set_params(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return {'loss': -accuracy_score(y_test, pred), 'status': STATUS_OK}

space = {'learning_rate': hp.uniform('learning_rate', 0.01, 0.3), 'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7]),
    'n_estimators': hp.choice('n_estimators', [50, 100, 150, 200]), 'subsample': hp.uniform('subsample', 0.6, 1.0),
    'reg_lamda':  hp.choice('reg_lambda', [10, 30, 60, 80, 100]), "reg_alpha": hp.choice('reg_alpha', [10, 30, 70, 110])}

In [43]:
trials = Trials()
start_time = time.time()
best = fmin(
    fn=partial(objective, XGBClassifier(random_state=13)), space=space, algo=tpe.suggest, max_evals=200, trials=trials)
hp_time = time.time() - start_time
best_params = {'learning_rate': best['learning_rate'], 'max_depth': [3,4,5,6,7][best['max_depth']],
    'n_estimators': [50,100,150,200][best['n_estimators']], 'subsample': best['subsample'],
    'reg_lambda': [10,30,60,80,100][best['reg_lambda']], 'reg_alpha': [10, 30, 70, 110][best['reg_alpha']]}
print("Лучшие параметры:", best_params)
print("Лучшая точность:", -trials.best_trial['result']['loss'])
print("Время", hp_time)

100%|██████████| 200/200 [00:07<00:00, 25.47trial/s, best loss: -0.8246753246753247]
Лучшие параметры: {'learning_rate': 0.2387991686875794, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.9220328558260146, 'reg_lambda': 10, 'reg_alpha': 10}
Лучшая точность: 0.8246753246753247
Время 7.858797788619995


Проанализируем полученные результаты.

Hyperopt TPE показал лучший результат по точности, чем случайный поиск. При этом случайный поиск проводится быстрее. При этом гиперпараметры склонны к следующему: learning_rate довольно больше 0.1-0.25, глубина деревьев может сильно колебаться, количество деревьев >= 100, reg_alpha маленький, reg_lambda может быть чуть больше, размер подвыборки должен быть >=0.7. В результате мы смогли получить качество, которое подбирали сами в задании 2 лабораторной 5.2.